# Speed of convergence of the Schwarz method with modified TBC

In this notebook, we will implement the Schwarz method with modified TBC conditions to solve the linearized KdV equation in a domain splitted in two, as described in the notebook TBC_besse_application.

The TBC and the proposed modifications depend on the coefficients $c_L$ and $c_R$ (the constant polynomial approximation proposed for the TBCs derived by Besse et al (2015)). After validating the modified TBCs with the pair of coefficients $(c_L,c_R) = (1.6,1.0)$ that minimized the error of the solution computed in the monodomain, we will now search the pair $(c_L,c_R)$ that produces the faster convergence.

As we done for the optimization process mentioned above, we will perform the computation with severals pairs $(c_L,c_R)$. In order to analyze exclusively the errors and the convergence behavior of the domain decomposition method (without influence of the time evolution), we will solve the problem along only one time step $\Delta t$. For each calculation, we will compute the number of iterations to reach some convergence criteria and the error $e(c_L,c_R)$ computed in the whole domain : 

$$e(c_L,c_R) = ||(u_1,u_2)(c_L,c_R) - u_{ref}(c_L,c_R)||$$

where $(u_1,u_2)$ is the vector containing the solutions $u_1$ and $u_2$ of each subdomain, and $u_{ref}$ is the reference solution (computed in the monodomain with the same coefficients $(c_L,c_R)$).

We will perform two series of tests, with different criteria of convergence that could be used in practice : 

* 1 : The method is stopped when the difference between two iterations, $max{||(u_1)^{k+1} - (u_1)^k||,||(u_2)^{k+1} - (u_2)^k||}$ gets smaller than a defined limit.
* 2 : The method is stopped when the difference of the solution in the interface point for $\Omega_1$ and $\Omega_2$ $\left(||(u_1)_N^{k+1} - (u_2)_N^k||\right)$ gets smaller than a defined limit.


In [1]:
import numpy as np
import kdv
import besseTBC
import matplotlib.pyplot as plt
import pickle
import generalFunctions as gF
import besseSchwarz as bS

%matplotlib inline

## First series of tests

Stop criteria : difference between two iterations

In [6]:
cLs = np.arange(0.3,2.25,0.05)
cRs = np.arange(0.3,2.25,0.05)

computeExactSolution = False

L = 6.
xmin = -L
xmax = L
N = 499
U2 = 1.
t0 = 0.

dt = 5*4/2560.
tmax = dt

dx = (xmax-xmin)/N
x = np.linspace(xmin,xmax,N)
xx = np.linspace(xmin,xmax,N+1)

dx = np.diff(x)[0]

x1 = x[0:x.size/2+1]
x2 = x[x.size/2:]

u = besseTBC.initGauss(x)
u1 = besseTBC.initGauss(x1)
u2 = besseTBC.initGauss(x2)

tallaux = np.arange(t0,tmax+dt,dt)

# Main

filename = 'speedTestsDiffIteration.json'
testsSpeed = besseTBC.loadTests(filename)

for cL in cLs:
    for cR in cRs:
        
        if (str((cL,cR)) not in testsSpeed.keys()) :
        

            coefTBC = np.zeros((1,2))
            coefTBC[0,0] = cL
            coefTBC[0,1] = cR

            ## Compute referential solution
            uallref,tallref = besseTBC.runDispKdV(x,u,t0,tmax,1., coefTBC , periodic = 0, vardt = False, dt = dt,
                                                          order = 0, modifyDiscret = 1)  

            nx = u1.size
            u1 = uallref[:nx,0]
            u2 = uallref[nx-1:,0]

            uall1Speed,uall2Speed,tall,niterallSpeed,diff1allSpeed,\
            diff2allSpeed,diffitallSpeed,errallSpeed,errIntLSpeed,errIntRSpeed = bS.runSimulation(x1,
                                                                                x2,u1,u2,t0,tmax,dx,dt,cL,cR,
                                                                               uref=uallref,maxiter = 1000,eps = 1e-7,
                                                                                printstep=100, debug=0,corrTBC=1, verbose=0,
                                                                                fourConditions = 0, pointR = 0)
            print(cL,cR,niterallSpeed[1],errallSpeed[-1,1])
            #if errallSpeed[-1,1] != 'nan' and  errallSpeed[-1,1] != 'inf' and errallSpeed[-1,1]  < 1e6:
            testsSpeed[str((cL,cR))] = (niterallSpeed[1],errallSpeed[-1,1])
                                
besseTBC.saveTests(testsSpeed,filename)

In [7]:
bS.showRankingSpeed(testsSpeed,20,criteria="speed")

Best results
('(0.000,0.000)', 3, 0.0044152593947872866)
('(0.650,1.400)', 8, 1.5450416275683102e-10)
('(0.650,1.400)', 8, 1.5448065743295838e-10)
('(0.850,1.550)', 8, 2.289705888752448e-10)
('(0.450,1.050)', 9, 1.3273277669732598e-08)
('(1.150,1.650)', 9, 2.8228505151164473e-08)
('(0.400,0.900)', 9, 2.3531291191571312e-08)
('(0.300,0.500)', 9, 4.519454844700106e-09)
('(0.350,0.700)', 9, 8.33472257472852e-08)
('(0.600,1.350)', 9, 5.5726298837254694e-08)
('(0.350,0.750)', 9, 2.192017846067233e-09)
('(1.150,1.650)', 9, 2.8228399181858716e-08)
('(0.300,0.550)', 9, 1.7812610861836198e-08)
('(1.450,1.700)', 9, 1.5407537360472682e-09)
('(0.400,0.900)', 9, 2.353132449222931e-08)
('(0.700,1.450)', 9, 2.772052379572924e-09)
('(0.300,0.350)', 11, 2.0140363274416388e-08)
('(0.550,1.250)', 11, 2.6115628681898846e-09)
('(2.200,1.750)', 11, 6.859845417886177e-09)
('(0.300,0.300)', 11, 1.0220591242503954e-08)


In [8]:
bS.showRankingSpeed(testsSpeed,20,criteria="error")

Best results
('(0.650,1.400)', 8, 1.5448065743295838e-10)
('(0.650,1.400)', 8, 1.5450416275683102e-10)
('(0.850,1.550)', 8, 2.289705888752448e-10)
('(1.450,1.700)', 9, 1.5407537360472682e-09)
('(0.350,0.750)', 9, 2.192017846067233e-09)
('(0.550,1.250)', 11, 2.6115628681898846e-09)
('(0.550,1.300)', 12, 2.7204177649404664e-09)
('(0.700,1.450)', 9, 2.772052379572924e-09)
('(1.000,1.600)', 12, 2.8633258122936035e-09)
('(0.450,1.000)', 12, 2.9679662950100566e-09)
('(1.500,1.700)', 12, 3.0854749470346746e-09)
('(0.300,0.450)', 11, 3.9369705502236064e-09)
('(1.100,1.650)', 12, 3.976589630884358e-09)
('(0.950,1.600)', 11, 4.22319436367565e-09)
('(2.150,1.750)', 12, 4.467631292161814e-09)
('(0.300,0.500)', 9, 4.519454844700106e-09)
('(0.400,0.950)', 11, 4.596591736297902e-09)
('(0.400,1.050)', 16, 4.697360635401558e-09)
('(0.350,0.900)', 16, 4.951679767754654e-09)
('(0.800,1.500)', 12, 5.632739376920893e-09)


## Second series of tests

Stop criteria : difference between the values on the interface

In [10]:
cLs = np.arange(0.0,2.25,0.05)
cRs = np.arange(0.0,2.25,0.05)

computeExactSolution = False

L = 6.
xmin = -L
xmax = L
N = 499
U2 = 1.
t0 = 0.

dt = 5*4/2560.
tmax = dt

dx = (xmax-xmin)/N
x = np.linspace(xmin,xmax,N)
xx = np.linspace(xmin,xmax,N+1)

dx = np.diff(x)[0]

x1 = x[0:x.size/2+1]
x2 = x[x.size/2:]

u = besseTBC.initGauss(x)
u1 = besseTBC.initGauss(x1)
u2 = besseTBC.initGauss(x2)

tallaux = np.arange(t0,tmax+dt,dt)

# Main

filename = 'speedTestsDiffInterface.json'
testsSpeed = besseTBC.loadTests(filename)

for cL in cLs:
    for cR in cRs:
        
        if (str((cL,cR)) not in testsSpeed.keys()) :
        

            coefTBC = np.zeros((1,2))
            coefTBC[0,0] = cL
            coefTBC[0,1] = cR

            ## Compute referential solution
            uallref,tallref = besseTBC.runDispKdV(x,u,t0,tmax,1., coefTBC , periodic = 0, vardt = False, dt = dt,
                                                          order = 0, modifyDiscret = 1)  

            nx = u1.size
            u1 = uallref[:nx,0]
            u2 = uallref[nx-1:,0]

            uall1Speed,uall2Speed,tall,niterallSpeed,diff1allSpeed,\
            diff2allSpeed,diffitallSpeed,errallSpeed,errIntLSpeed,errIntRSpeed = bS.runSimulation(x1,
                                                                                x2,u1,u2,t0,tmax,dx,dt,cL,cR,
                                                                               uref=uallref,maxiter = 1000,eps = 1e-7,
                                                                                printstep=100, debug=0,corrTBC=1, verbose=0,
                                                                                fourConditions = 0, pointR = 0)
            print(cL,cR,niterallSpeed[1],errallSpeed[-1,1])
            #if errallSpeed[-1,1] != 'nan' and  errallSpeed[-1,1] != 'inf' and errallSpeed[-1,1]  < 1e6:
            testsSpeed[str((cL,cR))] = (niterallSpeed[1],errallSpeed[-1,1])
                                
besseTBC.saveTests(testsSpeed,filename)

In [11]:
bS.showRankingSpeed(testsSpeed,20,criteria="speed")

Best results
('(0.000,0.000)', 1, 0.0042428420185223765)
('(1.000,0.250)', 3, 0.00020307844495694925)
('(0.400,0.900)', 6, 5.604174818780966e-07)
('(0.400,0.950)', 6, 1.8157432778734043e-06)
('(0.800,0.200)', 7, 7.412297821866151e-09)
('(0.350,0.200)', 7, 1.503966557061713e-08)
('(1.050,0.200)', 7, 6.734554706535815e-09)
('(0.600,0.200)', 7, 8.734463590126772e-09)
('(0.950,0.200)', 7, 6.939233116346338e-09)
('(1.850,0.200)', 7, 6.12835181878623e-09)
('(1.500,0.200)', 7, 6.2754745177883344e-09)
('(1.900,0.200)', 7, 6.113795596567378e-09)
('(1.300,0.200)', 7, 6.41971470526606e-09)
('(0.450,0.200)', 7, 1.1216853830628149e-08)
('(1.600,0.200)', 7, 6.223228832151062e-09)
('(2.100,0.200)', 7, 6.0657796563461185e-09)
('(0.400,0.200)', 7, 1.2765842459746932e-08)
('(0.650,0.200)', 7, 8.279383935233881e-09)
('(1.100,0.200)', 7, 6.653342935601465e-09)
('(1.750,0.200)', 7, 6.16135037918791e-09)


In [12]:
bS.showRankingSpeed(testsSpeed,20,criteria="error")

Best results
('(0.650,1.400)', 8, 1.5445902111000282e-10)
('(0.850,1.550)', 8, 2.2902920279521717e-10)
('(0.350,0.750)', 8, 9.557732422288934e-10)
('(1.400,1.700)', 12, 1.6492108873169119e-09)
('(0.550,1.250)', 10, 1.8298109768430585e-09)
('(1.450,1.700)', 8, 2.160374838335362e-09)
('(0.250,0.400)', 12, 2.3914605359681483e-09)
('(0.700,1.450)', 8, 2.4724847311962937e-09)
('(0.250,0.350)', 10, 2.5041114398552436e-09)
('(0.050,0.200)', 9, 2.6149980542221027e-09)
('(0.550,1.300)', 12, 2.7203940868582255e-09)
('(1.000,1.600)', 12, 2.863372427570136e-09)
('(0.300,0.500)', 8, 3.026905552395874e-09)
('(0.000,0.200)', 9, 3.028911172618716e-09)
('(0.300,0.450)', 10, 3.040073341342124e-09)
('(1.500,1.700)', 12, 3.0853700791522768e-09)
('(0.250,0.300)', 8, 3.3852380284185422e-09)
('(0.200,0.150)', 8, 3.508898879660243e-09)
('(0.750,1.450)', 14, 3.829599715512512e-09)
('(1.100,1.650)', 12, 3.976759669171148e-09)
